## Video Games Sales

`pip install pandas`

In [363]:

from datetime import datetime, timezone
import filecmp
import os
import pandas as pd

resources_path = "resources/"

vg_sales_path = resources_path + "vg_sales/"
vg_sales_res = {
    "v0": "vg_sales_v0.csv",
    "v1": "vg_sales_v1.csv",
    "v2": "vg_sales_v2.csv",
    "extended_release_year": "vg_sales_extended_release_year.csv",
    "extended_release_year_2": "vg_sales_extended_release_year_2.csv",
    "filled_year": "vg_sales_filled_year.csv",
    "filled_year_2": "vg_sales_filled_year_2.csv",
    "batch": "vg_sales_batches/"
}

vg_developers_path = resources_path + "vg_developers/"
vg_developers_res = {
    "v0": "vg_developers_v0.csv",
    "v1": "vg_developers_v1.csv",
}

vg_developers_association_path = resources_path + "vg_developers_association/"
vg_developers_association_res = {
    "v0": "vg_developers_association_v0.csv",
}


results_path = "results/"

In [ ]:
def save_to_csv(df, folder_path, filename):
    # Generate the full path for the new file
    new_file_path = os.path.join(folder_path, filename)
    
    # Check if the folder exists
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    # Save the dataframe to a temporary file
    temp_file_path = os.path.join(folder_path, 'temp_' + filename)
    df.to_csv(temp_file_path, index=True, index_label=df.index.name)
    
    # Compare the temporary file with existing files in the folder
    for existing_file in os.listdir(folder_path):
        existing_file_path = os.path.join(folder_path, existing_file)
        if existing_file_path != temp_file_path and filecmp.cmp(temp_file_path, existing_file_path, shallow=False):
            print(f"The content of {filename} already exists as {existing_file}. Not saving the file.")
            os.remove(temp_file_path)
            return
    
    # If no matching file is found, rename the temporary file to the new file
    os.rename(temp_file_path, new_file_path)
    print(f"File saved as {new_file_path}")

In [300]:
# Load the datasets with index

# vg_sales
vg_sales = pd.read_csv(vg_sales_path + vg_sales_res["v0"], index_col=0)

In [301]:
# columns
print("vg_sales.columns :", vg_sales.columns)

vg_sales.columns : Index(['Platform', 'Year_of_Release', 'Genre', 'Publisher', 'NA_Sales',
       'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales', 'Critic_Score',
       'Critic_Count', 'User_Score', 'User_Count', 'Developer', 'Rating'],
      dtype='object')


In [302]:
# NaN count
nan_count = vg_sales.isna().sum()
print(nan_count)

Platform              0
Year_of_Release     269
Genre                 2
Publisher            54
NA_Sales              0
EU_Sales              0
JP_Sales              0
Other_Sales           0
Global_Sales          0
Critic_Score       8582
Critic_Count       8582
User_Score         6704
User_Count         9129
Developer          6623
Rating             6769
dtype: int64


### Fix Year_Of_Release

In [303]:
# Create a folder in results_path if it doesn't exist
folder_name = 'vg_sales_no_year'
folder_path = os.path.join(results_path, folder_name)
os.makedirs(folder_path, exist_ok=True)

# Isolate rows where Year_of_Release is NaN
vg_sales_no_year = vg_sales[vg_sales['Year_of_Release'].isna()]

# Generate a unique filename based on the current date and time
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f'vg_sales_no_year_{timestamp}.csv'

# Save to a separate CSV file
save_to_csv(vg_sales_no_year, folder_path, filename)

The content of vg_sales_no_year_20241212_105144.csv already exists as vg_sales_no_year_20241210_162859.csv. Not saving the file.


In [304]:
# Load the extended release year data
vg_sales_extended = pd.read_csv(vg_sales_path + vg_sales_res["extended_release_year"], index_col=0)

# Merge the extended release year data with the original vg_sales dataframe
vg_sales = vg_sales.merge(vg_sales_extended[['Year_of_Release']], left_index=True, right_index=True, how='left', suffixes=('', '_extended'))

# Update the Year_of_Release column with the extended data where available
vg_sales['Year_of_Release'] = vg_sales['Year_of_Release_extended'].combine_first(vg_sales['Year_of_Release'])

# Drop the extended column as it's no longer needed
vg_sales.drop(columns=['Year_of_Release_extended'], inplace=True)

# Display the updated dataframe
print(vg_sales.head())

# Create a folder in results_path if it doesn't exist
folder_name = 'vg_sales_filled_year'
folder_path = os.path.join(results_path, folder_name)
os.makedirs(folder_path, exist_ok=True)

# Generate a unique filename based on the current date and time
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f'vg_sales_filled_year_{timestamp}.csv'

# Save the updated dataframe to a separate CSV file
save_to_csv(vg_sales, folder_path, filename)

                         Platform  Year_of_Release         Genre Publisher  \
Name                                                                         
Wii Sports                    Wii           2006.0        Sports  Nintendo   
Super Mario Bros.             NES           1985.0      Platform  Nintendo   
Mario Kart Wii                Wii           2008.0        Racing  Nintendo   
Wii Sports Resort             Wii           2009.0        Sports  Nintendo   
Pokemon Red/Pokemon Blue       GB           1996.0  Role-Playing  Nintendo   

                          NA_Sales  EU_Sales  JP_Sales  Other_Sales  \
Name                                                                  
Wii Sports                   41.36     28.96      3.77         8.45   
Super Mario Bros.            29.08      3.58      6.81         0.77   
Mario Kart Wii               15.68     12.76      3.79         3.29   
Wii Sports Resort            15.61     10.93      3.28         2.95   
Pokemon Red/Pokemon Blue   

In [305]:
# vg_sales_filled_year
vg_sales_filled_year = pd.read_csv(vg_sales_path + vg_sales_res["filled_year"], index_col=0)

In [306]:
# Create a folder in results_path if it doesn't exist
folder_name = 'vg_sales_filled_year_no_year'
folder_path = os.path.join(results_path, folder_name)
os.makedirs(folder_path, exist_ok=True)

# Isolate rows where Year_of_Release is NaN
vg_sales_filled_year_no_year = vg_sales_filled_year[vg_sales_filled_year['Year_of_Release'].isna()]

# Generate a unique filename based on the current date and time
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f'vg_sales_filled_year_no_year_{timestamp}.csv'

# Save to a separate CSV file
save_to_csv(vg_sales_filled_year_no_year, folder_path, filename)

The content of vg_sales_filled_year_no_year_20241212_105147.csv already exists as vg_sales_filled_year_no_year_20241210_162927.csv. Not saving the file.


In [307]:
# Load the extended release year data
vg_sales_extended_2 = pd.read_csv(vg_sales_path + vg_sales_res["extended_release_year_2"], index_col=0)

# Make a copy of vg_sales before making any changes
vg_sales_v0_5 = vg_sales.copy()

# Merge the extended release year data with the copied vg_sales dataframe
vg_sales_v0_5 = vg_sales_v0_5.merge(vg_sales_extended_2[['Year_of_Release']], left_index=True, right_index=True, how='left', suffixes=('', '_extended'))

# Update the Year_of_Release column with the extended data where available
vg_sales_v0_5['Year_of_Release'] = vg_sales_v0_5['Year_of_Release_extended'].combine_first(vg_sales_v0_5['Year_of_Release'])

# Drop the extended column as it's no longer needed
vg_sales_v0_5.drop(columns=['Year_of_Release_extended'], inplace=True)

# Display the updated dataframe
print(vg_sales_v0_5.head())

# Create a folder in results_path if it doesn't exist
folder_name = 'vg_sales_filled_year_2'
folder_path = os.path.join(results_path, folder_name)
os.makedirs(folder_path, exist_ok=True)

# Generate a unique filename based on the current date and time
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f'vg_sales_filled_year_2_{timestamp}.csv'

# Save the updated dataframe to a separate CSV file
save_to_csv(vg_sales_v0_5, folder_path, filename)

                         Platform  Year_of_Release         Genre Publisher  \
Name                                                                         
Wii Sports                    Wii           2006.0        Sports  Nintendo   
Super Mario Bros.             NES           1985.0      Platform  Nintendo   
Mario Kart Wii                Wii           2008.0        Racing  Nintendo   
Wii Sports Resort             Wii           2009.0        Sports  Nintendo   
Pokemon Red/Pokemon Blue       GB           1996.0  Role-Playing  Nintendo   

                          NA_Sales  EU_Sales  JP_Sales  Other_Sales  \
Name                                                                  
Wii Sports                   41.36     28.96      3.77         8.45   
Super Mario Bros.            29.08      3.58      6.81         0.77   
Mario Kart Wii               15.68     12.76      3.79         3.29   
Wii Sports Resort            15.61     10.93      3.28         2.95   
Pokemon Red/Pokemon Blue   

In [308]:
# vg_sales_filled_year_2
vg_sales_filled_year_2 = pd.read_csv(vg_sales_path + vg_sales_res["filled_year_2"], index_col=0)

In [309]:
# Create a folder in results_path if it doesn't exist
folder_name = 'vg_sales_filled_year_2_no_year'
folder_path = os.path.join(results_path, folder_name)
os.makedirs(folder_path, exist_ok=True)

# Isolate rows where Year_of_Release is NaN
vg_sales_filled_year_2_no_year = vg_sales_filled_year_2[vg_sales_filled_year_2['Year_of_Release'].isna()]

# Generate a unique filename based on the current date and time
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f'vg_sales_filled_year_2_no_year_{timestamp}.csv'

# Save to a separate CSV file
save_to_csv(vg_sales_filled_year_2_no_year, folder_path, filename)

The content of vg_sales_filled_year_2_no_year_20241212_105153.csv already exists as vg_sales_filled_year_2_no_year_20241210_165438.csv. Not saving the file.


### Cancelled games in the dataset
Brothers in Arms: Furious 4,X360,,Shooter,,0.01,0.0,0.0,0.0,0.01,,,,,Gearbox Software,M
was never released and put into sales

In [310]:
# Remove the specified entry
vg_sales_cleaned_release_date = vg_sales_filled_year_2.drop(index='Brothers in Arms: Furious 4') # type: ignore

# Create a folder in results_path if it doesn't exist
folder_name = 'vg_sales_cleaned_release_date'
folder_path = os.path.join(results_path, folder_name)
os.makedirs(folder_path, exist_ok=True)

# Generate a unique filename based on the current date and time
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f'vg_sales_cleaned_release_date_{timestamp}.csv'

# Save the cleaned dataframe to a separate CSV file
save_to_csv(vg_sales_cleaned_release_date, folder_path, filename)

The content of vg_sales_cleaned_release_date_20241212_105154.csv already exists as vg_sales_cleaned_release_date_20241210_170408.csv. Not saving the file.


In [311]:
# vg_sales_v1
vg_sales_v1 = pd.read_csv(vg_sales_path + vg_sales_res["v1"], index_col=0)

In [312]:
# Isolate rows where Year_of_Release is NaN
vg_sales_v1_no_release_date = vg_sales_v1[vg_sales_v1['Year_of_Release'].isna()]
print("Number of video games that have no release date :", len(vg_sales_v1_no_release_date))

Number of video games that have no release date : 0


### Fix Developer & Rating

In [313]:

import pandas as pd
import requests
import time
from tqdm.notebook import tqdm

# Mapping ratings to letters
rating_map = {
    1: "Three",
    2: "Seven",
    3: "Twelve",
    4: "Sixteen",
    5: "Eighteen",
    6: "RP",
    7: "EC",
    8: "E",
    9: "E10",
    10: "T",
    11: "M",
    12: "AO",
    13: "CERO_A",
    14: "CERO_B",
    15: "CERO_C",
    16: "CERO_D",
    17: "CERO_Z",
    18: "USK_0",
    19: "USK_6",
    20: "USK_12",
    21: "USK_16",
    22: "USK_18",
    23: "GRAC_ALL",
    24: "GRAC_Twelve",
    25: "GRAC_Fifteen",
    26: "GRAC_Eighteen",
    27: "GRAC_TESTING",
    28: "CLASS_IND_L",
    29: "CLASS_IND_Ten",
    30: "CLASS_IND_Twelve",
    31: "CLASS_IND_Fourteen",
    32: "CLASS_IND_Sixteen",
    33: "CLASS_IND_Eighteen",
    34: "ACB_G",
    35: "ACB_PG",
    36: "ACB_M",
    37: "ACB_MA15",
    38: "ACB_R18",
    39: "ACB_RC"
}

unique_platforms = vg_sales_v1['Platform'].unique()
print(unique_platforms)

['Wii' 'NES' 'GB' 'DS' 'X360' 'PS3' 'PS2' 'SNES' 'GBA' 'PS4' '3DS' 'N64'
 'PS' 'XB' 'PC' '2600' 'PSP' 'XOne' 'WiiU' 'GC' 'GEN' 'DC' 'PSV' 'SAT'
 'SCD' 'WS' 'NG' 'TG16' '3DO' 'GG' 'PCFX']


In [314]:
# Function to get the access token from IGDB
def get_access_token(client_id, client_secret):
    url = "https://id.twitch.tv/oauth2/token"
    params = {
        "client_id": client_id,
        "client_secret": client_secret,
        "grant_type": "client_credentials"
    }
    response = requests.post(url, params=params)
    return response.json()['access_token']

# Your actual IGDB client ID and client secret
client_id = 'ms8zqnreo7k8v5v4fp198eivs5039o'
client_secret = 'ia642tpku45i8qwcmp1pvo4ybi838d'

# Get the access token
access_token = get_access_token(client_id, client_secret)

In [315]:
# Function to get game details from IGDB
def get_game_details(game_name, release_year, platform, client_id, access_token):
    url = "https://api.igdb.com/v4/games"
    headers = {
        "Client-ID": client_id,
        "Authorization": f"Bearer {access_token}"
    }
    platform_ids = {
        "PC": 6,
        "Wii": 5,
        "NES": 18,
        "X360": 12,
        "PS3": 9,
        "PS2": 8,
        "SNES": 19,
        "PS4": 48,
        "N64": 4,
        "PS": 7,
        "XB": 11,
        "2600": 59,
        "XOne": 49,
        "WiiU": 41,
        "GC": 21,
        "GEN": 29,
        "DC": 23,
        "SAT": 32,
        "SCD": 22,
        "3DO": 50,
        "GB": 33,
        "DS": 20,
        "GBA": 24,
        "3DS": 37,
        "PSP": 38,
        "PSV": 46,
        "WS": 35,
        "NG": 119,
        "TG16": 86,
        "GG": 87,
        "PCFX": 88
    }
    platform_id = platform_ids.get(platform, "unknown")
    if platform_id != "unknown":
        data = f'search "{game_name}"; where release_dates.y = {int(release_year)} & release_dates.platform = {platform_id}; fields name, involved_companies.company.name, age_ratings.rating;'
    else:
        data = f'search "{game_name}"; where release_dates.y = {int(release_year)}; fields name, involved_companies.company.name, age_ratings.rating;'
    response = requests.post(url, headers=headers, data=data)
    return response.json()

# Example usage
game_name = "Super Mario Bros."
release_year = 1985
platform = "NES"

game_details = get_game_details(game_name, release_year, platform, client_id, access_token)
print(game_details)

[{'id': 358, 'age_ratings': [{'id': 24, 'rating': 8}, {'id': 33840, 'rating': 1}, {'id': 47142, 'rating': 28}, {'id': 47471, 'rating': 13}, {'id': 50767, 'rating': 18}, {'id': 50768, 'rating': 23}, {'id': 50769, 'rating': 34}], 'involved_companies': [{'id': 225308, 'company': {'id': 33804, 'name': 'Playtronic'}}, {'id': 256833, 'company': {'id': 70, 'name': 'Nintendo'}}, {'id': 256834, 'company': {'id': 32471, 'name': 'Nintendo R&D4'}}, {'id': 256846, 'company': {'id': 70, 'name': 'Nintendo'}}, {'id': 256847, 'company': {'id': 32471, 'name': 'Nintendo R&D4'}}, {'id': 256939, 'company': {'id': 70, 'name': 'Nintendo'}}, {'id': 256940, 'company': {'id': 32471, 'name': 'Nintendo R&D4'}}], 'name': 'Super Mario Bros.'}]


In [316]:
# Load country mapping from iso_3166_countries.csv
iso_3166_countries_path = os.path.join(resources_path, "iso_3166_countries.csv")
country_mapping_df = pd.read_csv(iso_3166_countries_path, usecols=['name', 'country-code', 'region'])

# Rename columns for better readability
country_mapping_df.rename(columns={'name': 'Country', 'country-code': 'Country_Code', 'region': 'Region'}, inplace=True)

# Convert the DataFrame to a dictionary
COUNTRY_MAPPING = country_mapping_df.set_index('Country_Code')['Country'].to_dict()

print(COUNTRY_MAPPING)

{4: 'Afghanistan', 248: 'Åland Islands', 8: 'Albania', 12: 'Algeria', 16: 'American Samoa', 20: 'Andorra', 24: 'Angola', 660: 'Anguilla', 10: 'Antarctica', 28: 'Antigua and Barbuda', 32: 'Argentina', 51: 'Armenia', 533: 'Aruba', 36: 'Australia', 40: 'Austria', 31: 'Azerbaijan', 44: 'Bahamas', 48: 'Bahrain', 50: 'Bangladesh', 52: 'Barbados', 112: 'Belarus', 56: 'Belgium', 84: 'Belize', 204: 'Benin', 60: 'Bermuda', 64: 'Bhutan', 68: 'Bolivia, Plurinational State of', 535: 'Bonaire, Sint Eustatius and Saba', 70: 'Bosnia and Herzegovina', 72: 'Botswana', 74: 'Bouvet Island', 76: 'Brazil', 86: 'British Indian Ocean Territory', 96: 'Brunei Darussalam', 100: 'Bulgaria', 854: 'Burkina Faso', 108: 'Burundi', 132: 'Cabo Verde', 116: 'Cambodia', 120: 'Cameroon', 124: 'Canada', 136: 'Cayman Islands', 140: 'Central African Republic', 148: 'Chad', 152: 'Chile', 156: 'China', 162: 'Christmas Island', 166: 'Cocos (Keeling) Islands', 170: 'Colombia', 174: 'Comoros', 178: 'Congo', 180: 'Congo, Democrati

In [317]:
def convert_unix_to_year(unix_timestamp):
    if unix_timestamp is None:
        return "Unknown"
    try:
        # Use timezone.utc correctly for UTC-based conversion
        dt = datetime.fromtimestamp(unix_timestamp, tz=timezone.utc)
        return dt.year
    except (ValueError, OSError, TypeError):
        # Handle invalid timestamp values
        return "Unknown"

In [318]:
# Function to get company details from IGDB
def get_company_details(company_name, client_id, access_token):
    url = "https://api.igdb.com/v4/companies"
    headers = {
        "Client-ID": client_id,
        "Authorization": f"Bearer {access_token}"
    }
    data = f'fields id, name, country, start_date; where name ~ "{company_name}";'
    response = requests.post(url, headers=headers, data=data)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}, {response.text}")
        return None

    companies = response.json()
    parsed_companies = []

    for company in companies:
        country_id = company.get('country')
        start_date = company.get('start_date')
        parsed_company = {
            "id": company.get('id'),
            "name": company.get('name'),
            "country": COUNTRY_MAPPING.get(country_id, f"Unknown (ID: {country_id})"),
            "start_year": convert_unix_to_year(start_date),
        }
        parsed_companies.append(parsed_company)
    
    return parsed_companies

# Example usage
company_name = "Nintendo"
company_details = get_company_details(company_name, client_id, access_token)
print(company_details)


[{'id': 70, 'name': 'Nintendo', 'country': 'Japan', 'start_year': 1889}]


In [319]:
# Drop 'Developer' and 'Rating' columns
vg_sales_v1_5 = vg_sales_v1.drop(columns=['Developer', 'Rating'])

In [320]:
# Initialize an empty DataFrame for developers
developers_df = pd.DataFrame(columns=['Developer', 'Country', 'Year_of_Establishment'])
developers_df.set_index('Developer', inplace=True)
developers_df.index.name = 'Developer'

# Initialize an empty DataFrame for developers and video games association
developers_vg_association_df = pd.DataFrame(columns=['Developer', 'Video_Game_Name'])

In [321]:
# Update the DataFrame with a progress bar and save added details in a separate DataFrame
def update_game_info_with_progress(df, client_id, access_token, developers_df, developers_vg_association_df, video_games_sales=None):
    request_count = 0
    added_details = []

    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Updating game info", leave=True):
        game_name = row.name
        updated_row = row.copy()

        try:
            game_details = get_game_details(game_name, row['Year_of_Release'], row['Platform'], client_id, access_token)
        except Exception as e:
            print(f"Error fetching details for {game_name}: {e}")
            continue
        
        if game_details and isinstance(game_details, list) and len(game_details) > 0:
            game_info = game_details[0]
            if 'involved_companies' in game_info and game_info['involved_companies']:
                involved_companies = set()
                for company in game_info['involved_companies']:
                    developer_name = company['company']['name']
                    
                    if developer_name not in involved_companies:
                        involved_companies.add(developer_name)
                        
                        # Add developer to developers_df if not exists
                        if developer_name not in developers_df.index:
                            company_details = get_company_details(developer_name, client_id, access_token)
                            if company_details:
                                company_info = company_details[0]
                                developers_df.loc[developer_name] = {
                                    'Country': company_info['country'],
                                    'Year_of_Establishment': company_info['start_year']
                                }

                        # Add association to developers_vg_association_df
                        new_row = pd.DataFrame([{'Developer': developer_name, 'Video_Game_Name': game_name}])
                        developers_vg_association_df = pd.concat([developers_vg_association_df, new_row])
                        print(f"Added association: Developer={developer_name}, Game={game_name}")

            if 'age_ratings' in game_info and game_info['age_ratings']:
                numeric_rating = game_info['age_ratings'][0]['rating']
                updated_row['Rating'] = rating_map.get(numeric_rating, "Unknown")
                print(f"Updated {game_name}: Rating={updated_row['Rating']}")

            # Rate limiting
            request_count += 1
            if request_count >= 4:
                time.sleep(1)
                request_count = 0

        added_details.append(updated_row)

    added_details_df = pd.DataFrame(added_details)
    return added_details_df, developers_vg_association_df.reset_index(drop=True)


In [260]:
batch_size = 1000
num_batches = (len(vg_sales_v1_5) // batch_size) + 1
all_added_details = []
all_developers_vg_association = []

# Create a folder in results_path if it doesn't exist
folder_name = 'vg_sales_batches'
folder_path = os.path.join(results_path, folder_name)
os.makedirs(folder_path, exist_ok=True)

for i in range(num_batches):
    batch_df = vg_sales_v1_5.iloc[i * batch_size:(i + 1) * batch_size]
    print(f"Processing batch {i + 1}/{num_batches} with {len(batch_df)} records")
    batch_added_details_df, batch_developers_vg_association_df = update_game_info_with_progress(batch_df, client_id, access_token, developers_df, developers_vg_association_df, vg_sales)
    
    all_added_details.append(batch_added_details_df)
    all_developers_vg_association.append(batch_developers_vg_association_df)
    
    # Create a folder for each batch
    batch_folder_name = f'vg_sales_batch_{i + 1}'
    batch_folder_path = os.path.join(folder_path, batch_folder_name)
    os.makedirs(batch_folder_path, exist_ok=True)
    
    # Generate a unique filename for each batch based on the current date and time
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    batch_filename = f'vg_sales_batch_{i + 1}_{timestamp}.csv'
    
    batch_added_details_df.index.name = 'Name'
    
    # Save each batch to a separate CSV file
    save_to_csv(batch_added_details_df, batch_folder_path, batch_filename)
    
    # Save the developers_df and developers_vg_association_df for checking purposes
    save_to_csv(developers_df, batch_folder_path, f'developers_batch_{i + 1}_{timestamp}.csv')
    
    batch_developers_vg_association_df.index.name = 'Index'
    
    save_to_csv(batch_developers_vg_association_df, batch_folder_path, f'developers_vg_association_batch_{i + 1}_{timestamp}.csv')

Processing batch 1/17 with 1000 records


Updating game info:   0%|          | 0/1000 [00:00<?, ?it/s]

Added association: Developer=Nintendo, Game=Wii Sports
Added association: Developer=Nintendo EAD Software Development Group No.2, Game=Wii Sports
Updated Wii Sports: Rating=E
Added association: Developer=Playtronic, Game=Super Mario Bros.
Added association: Developer=Nintendo, Game=Super Mario Bros.
Added association: Developer=Nintendo R&D4, Game=Super Mario Bros.
Updated Super Mario Bros.: Rating=E
Added association: Developer=Nintendo, Game=Mario Kart Wii
Added association: Developer=Nintendo EAD, Game=Mario Kart Wii
Updated Mario Kart Wii: Rating=E
Added association: Developer=Nintendo, Game=Wii Sports Resort
Added association: Developer=Nintendo EAD Software Development Group No.2, Game=Wii Sports Resort
Updated Wii Sports Resort: Rating=E
Added association: Developer=Nintendo, Game=Tetris
Added association: Developer=Gradiente, Game=Tetris
Added association: Developer=Playtronic, Game=Tetris
Added association: Developer=ELORG, Game=Tetris
Added association: Developer=Mani, Game=T

Updating game info:   0%|          | 0/1000 [00:00<?, ?it/s]

Added association: Developer=Nintendo, Game=Metroid II: Return of Samus
Added association: Developer=Nintendo R&D1, Game=Metroid II: Return of Samus
Added association: Developer=Playtronic, Game=Metroid II: Return of Samus
Updated Metroid II: Return of Samus: Rating=Seven
Added association: Developer=THQ, Game=UFC 2009 Undisputed
Updated UFC 2009 Undisputed: Rating=T
Added association: Developer=Heavy Iron Studios, Game=The SpongeBob SquarePants Movie
Added association: Developer=THQ, Game=The SpongeBob SquarePants Movie
Updated The SpongeBob SquarePants Movie: Rating=E
Added association: Developer=Nintendo, Game=Golden Sun
Added association: Developer=Camelot Software Planning, Game=Golden Sun
Updated Golden Sun: Rating=E
Added association: Developer=Sonic Team, Game=Sonic the Hedgehog 3
Added association: Developer=Sega, Game=Sonic the Hedgehog 3
Added association: Developer=Tec Toy, Game=Sonic the Hedgehog 3
Added association: Developer=Sega Technical Institute, Game=Sonic the Hedge

...

Updated The Chase: Felix Meets Felicity: Rating=Three
Added association: Developer=LEVEL-5, Game=Time Travelers
Updated Time Travelers: Rating=CERO_C
Added association: Developer=Capcom, Game=Dead Rising
Added association: Developer=Capcom Production Studio 1, Game=Dead Rising
Added association: Developer=QLOC, Game=Dead Rising
Updated Dead Rising: Rating=M
Added association: Developer=Rockstar Games, Game=Grand Theft Auto: Vice City
Added association: Developer=Rockstar North, Game=Grand Theft Auto: Vice City
Added association: Developer=Rockstar Vienna, Game=Grand Theft Auto: Vice City
Added association: Developer=Take-Two Interactive, Game=Grand Theft Auto: Vice City
Added association: Developer=TransGaming Inc., Game=Grand Theft Auto: Vice City
Updated Grand Theft Auto: Vice City: Rating=Eighteen
Added association: Developer=Electronic Arts, Game=Syndicate Wars
Added association: Developer=Bullfrog Productions, Game=Syndicate Wars
Updated Sonic PC Collection: Rating=ACB_G
Added ass

Updating game info:   0%|          | 0/1000 [00:00<?, ?it/s]

Added association: Developer=Otomate, Game=AMNESIA World
Added association: Developer=Design Factory, Game=AMNESIA World
Added association: Developer=Idea Factory, Game=AMNESIA World
Updated AMNESIA World: Rating=CERO_B
Added association: Developer=Graphic Research, Game=Runabout 3: Neo Age
Added association: Developer=D3 Publisher, Game=Runabout 3: Neo Age
Added association: Developer=Focus Entertainment, Game=The Technomancer
Added association: Developer=Spiders, Game=The Technomancer
Updated The Technomancer: Rating=Sixteen
Added association: Developer=Bandai Namco Entertainment Asia, Game=Teddy Together
Added association: Developer=Nintendo of Europe, Game=Teddy Together
Added association: Developer=Arika, Game=Teddy Together
Updated Teddy Together: Rating=Three
Added association: Developer=ArenaNet, Game=Guild Wars: Nightfall
Added association: Developer=NCSOFT, Game=Guild Wars: Nightfall
Updated Guild Wars: Nightfall: Rating=T
Added association: Developer=Paradigm Entertainment, 

Updating game info:   0%|          | 0/854 [00:00<?, ?it/s]

Added association: Developer=Hudson Soft, Game=Bomberman 2
Updated Bomberman 2: Rating=Seven
Added association: Developer=MacSoft Games, Game=Unreal Tournament 2004
Added association: Developer=Digital Extremes, Game=Unreal Tournament 2004
Added association: Developer=Psyonix, Game=Unreal Tournament 2004
Added association: Developer=Epic Games, Game=Unreal Tournament 2004
Added association: Developer=Streamline Studios, Game=Unreal Tournament 2004
Added association: Developer=Atari, Inc., Game=Unreal Tournament 2004
Updated Unreal Tournament 2004: Rating=M
Added association: Developer=D3 Publisher, Game=Cube
Added association: Developer=Metia Interactive, Game=Cube
Updated Cube: Rating=E
Added association: Developer=Flying Lab Software, Game=Pirates of the Burning Sea
Added association: Developer=Sony Online Entertainment, Game=Pirates of the Burning Sea
Added association: Developer=Portalus Games, Game=Pirates of the Burning Sea
Updated Pirates of the Burning Sea: Rating=T
Added assoc

In [322]:
# Merge all batches into a single DataFrame
final_added_details_df = pd.concat(all_added_details)
final_developers_vg_association_df = pd.concat(all_developers_vg_association)

# Ensure the index names are correct
final_added_details_df.index.name = 'Name'
final_developers_vg_association_df.index.name = 'Index'

In [323]:
# Create a folder in results_path if it doesn't exist
folder_name = 'vg_sales_v2'
folder_path = os.path.join(results_path, folder_name)
os.makedirs(folder_path, exist_ok=True)

# Generate a unique filename based on the current date and time
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f'vg_sales_v2_{timestamp}.csv'

# Save the final DataFrame to a CSV file
save_to_csv(final_added_details_df, folder_path, filename)

# Save the final developers_df and developers_vg_association_df
save_to_csv(developers_df, folder_path, f'developers_{timestamp}.csv')
save_to_csv(final_developers_vg_association_df, folder_path, f'developers_vg_association_{timestamp}.csv')

# Display the final DataFrame
print(final_added_details_df.head())

# Save the final DataFrames as vg_sales_v2, developers_v1, and dev_vg_association_v1
vg_sales_v2 = final_added_details_df
developers_v1 = developers_df
dev_vg_association_v1 = developers_vg_association_df

The content of vg_sales_v2_20241212_105223.csv already exists as vg_sales_v2_20241212_103814.csv. Not saving the file.
The content of developers_20241212_105223.csv already exists as developers_20241212_105006.csv. Not saving the file.
The content of developers_vg_association_20241212_105223.csv already exists as developers_vg_association_20241212_103814.csv. Not saving the file.
                         Platform  Year_of_Release         Genre Publisher  \
Name                                                                         
Wii Sports                    Wii           2006.0        Sports  Nintendo   
Super Mario Bros.             NES           1985.0      Platform  Nintendo   
Mario Kart Wii                Wii           2008.0        Racing  Nintendo   
Wii Sports Resort             Wii           2009.0        Sports  Nintendo   
Pokemon Red/Pokemon Blue       GB           1996.0  Role-Playing  Nintendo   

                          NA_Sales  EU_Sales  JP_Sales  Other_Sales  \

In [333]:
vg_sales_v0 = pd.read_csv(vg_sales_path + vg_sales_res["v0"], index_col=0)

vg_sales_v2 = pd.read_csv(vg_sales_path + vg_sales_res["v2"], index_col=0)

print("--------- FROM ---------\n")

# NaN count vg_sales_v0
nan_count = vg_sales_v0.isna().sum()
print(nan_count)

print("\n--------- TO ---------\n")

# NaN count vg_sales_v2
nan_count = vg_sales_v2.isna().sum()
print(nan_count)

--------- FROM ---------

Platform              0
Year_of_Release     269
Genre                 2
Publisher            54
NA_Sales              0
EU_Sales              0
JP_Sales              0
Other_Sales           0
Global_Sales          0
Critic_Score       8582
Critic_Count       8582
User_Score         6704
User_Count         9129
Developer          6623
Rating             6769
dtype: int64

--------- TO ---------

Platform              0
Year_of_Release       0
Genre                 2
Publisher            53
NA_Sales              0
EU_Sales              0
JP_Sales              0
Other_Sales           0
Global_Sales          0
Critic_Score       8608
Critic_Count       8608
User_Score         6718
User_Count         9164
Rating             4789
dtype: int64


In [341]:
# Load the vg_developers_association dataset
vg_developers_association = pd.read_csv(vg_developers_association_path + vg_developers_association_res["v0"], index_col=0)

# Ensure the 'Video_Game_Name' column is correctly referenced
no_developer_games = vg_sales_v2[~vg_sales_v2.index.isin(vg_developers_association['Video_Game_Name'])]

# Count the number of games without developer details
no_developer_count = no_developer_games.shape[0]
print(f"Number of games without developer details: {no_developer_count}")

Number of games without developer details: 4146


In [342]:
# Check for non-null developers in no_developer_games within vg_sales_v0
no_developer_games_with_developer = no_developer_games.index[no_developer_games.index.isin(vg_sales_v0[vg_sales_v0['Developer'].notnull()].index)]

# Display the results
print(f"Number of games in no_developer_games that have a non-null developer in vg_sales_v0: {len(no_developer_games_with_developer)}")

Number of games in no_developer_games that have a non-null developer in vg_sales_v0: 1167


### Improvements

Developers can be added even further but with no extra details. Would necessitate more scrapping!

## Cleaning

In [364]:
vg_developers_v0_5 = pd.read_csv(vg_developers_path + vg_developers_res["v0"], index_col=0)

# Replace "Unknown (ID: None)" and "Unknown" with "Unknown"
vg_developers_v0_5.replace({"Unknown (ID: None)": "Unknown"}, inplace=True)

# Save the updated DataFrame to a CSV file
save_to_csv(vg_developers_v0_5, vg_developers_path, vg_developers_res["v1"])

The content of vg_developers_v1.csv already exists as vg_developers_v2.csv. Not saving the file.
